# Importação de Bibliotecas

In [1]:
# Python versão 3.8.8
# Windows 10

# biblioteca para interação com o sistema
import os
import shutil

# biblioteca para manipulação de dados
import pandas as pd # versão 1.3.2
import numpy as np # versão 1.20.1

# biblioteca para manipulação de banco de dados sqlite
import sqlite3 # versão 2.6.0
from sqlite3 import Error

import researchpy as rp

# biblioteca para visualização
import plotly.express as px # versão 5.3.1
# import seaborn as sns # versão 0.11.1

import plotly.express as px
import plotly.figure_factory as ff

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

import matplotlib.pyplot as plt  # versão 3.3.4

# configuracoes 
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Variáveis Globais

In [2]:
# Endereço do diretório onde se encontra o script python
diretorio_atual = os.path.dirname(os.path.realpath('__file__')) + "\\"

# Nome do Banco de Dados de onde serão importadas as tabelas
nome_bd = "Enade_DW.sqlite"

# Importação Tabelas do Banco de Dados

In [3]:
def nova_conexao(path, nome_bd):

    conec_bd = None
    try:
        conec_bd = sqlite3.connect(path + nome_bd)

    except Error as erro:
        print(erro)
        return 0
    
    finally:
        if conec_bd:
            conec_bd.close()
    return 1

In [4]:
# Ininia a Conexão
if (nova_conexao(diretorio_atual,  nome_bd)):
    conec_bd = sqlite3.connect(diretorio_atual + nome_bd)
    cursor = conec_bd.cursor()
    print("Conexão ao banco '{}' criada com sucesso.\n".format(nome_bd))
    print("-" * 90)
    conec_bd.commit()

# Importa tabelas do Banco de Dados
print(f"Importando tabelas do Banco de Dados{nome_bd}...", end="\r", flush=True)

dim_aluno_econ = pd.read_sql_query('SELECT * FROM DIM_ALUNO_ECON ', conec_bd, index_col ='ID_ALUNO_ECONOMICO')
dim_curso = pd.read_sql_query('SELECT * FROM DIM_CURSO ', conec_bd, index_col ='ID_CURSO')
dim_ensino_medio = pd.read_sql_query('SELECT * FROM DIM_ENSINO_MEDIO ', conec_bd, index_col ='ID_ALUNO_ENSINO_MEDIO')
dim_formacao_familiares = pd.read_sql_query('SELECT * FROM DIM_FORMACAO_FAMILIARES ', conec_bd, index_col ='ID_FORMACAO_FAMILIA')
dim_ies = pd.read_sql_query('SELECT * FROM DIM_IES ', conec_bd, index_col ='ID_IES_ANO')
dim_info_aluno = pd.read_sql_query('SELECT * FROM DIM_INFO_ALUNO ', conec_bd, index_col ='ID_ALUNO_INFO')
dim_oportunidades = pd.read_sql_query('SELECT * FROM DIM_OPORTUNIDADES ', conec_bd, index_col ='ID_OPORTUNIDADES')
dim_percepcao_prova = pd.read_sql_query('SELECT * FROM DIM_PERCEPCAO_PROVA ', conec_bd, index_col ='ID_PERCEPCAO_PROVA')
dim_presenca = pd.read_sql_query('SELECT * FROM DIM_PRESENCA ', conec_bd, index_col ='ID_PRESENCA')
dim_regiao = pd.read_sql_query('SELECT * FROM DIM_REGIAO ', conec_bd, index_col ='CO_MUNIC_CURSO')
dim_tempo = pd.read_sql_query('SELECT * FROM DIM_TEMPO ', conec_bd, index_col ='ID_TEMPO')
fato_prova = pd.read_sql_query('SELECT * FROM FATO_PROVA ', conec_bd, index_col ='ID_PROVA')

print(f"Tabelas importadas com Sucesso.\n                     ")
print("-" * 90)

# Encerra a Conexão
conec_bd.close()
print(f"Conexão ao banco '{nome_bd}' encerrada com sucesso.\n")

Conexão ao banco 'Enade_DW.sqlite' criada com sucesso.

------------------------------------------------------------------------------------------
Tabelas importadas com Sucesso.DadosEnade_DW.sqlite...
                     
------------------------------------------------------------------------------------------
Conexão ao banco 'Enade_DW.sqlite' encerrada com sucesso.



# Análise de Dados

In [5]:
df_aluno_info = dim_info_aluno.join(fato_prova.set_index("ID_ALUNO_INFO")[['NT_GER', 
                                                                           'NT_FG','NT_CE']])


In [6]:
df_aluno_info["NT_GER_INTEIRO"] = df_aluno_info["NT_GER"].astype(int)

## SEXO

In [7]:
df_aluno_info.TP_SEXO = df_aluno_info.TP_SEXO.str.replace("F", "Feminino")
df_aluno_info.TP_SEXO = df_aluno_info.TP_SEXO.str.replace("M", "Masculino")

In [8]:
rp.summary_cont(df_aluno_info['NT_GER'].groupby(df_aluno_info['TP_SEXO']))

,N,Mean,SD,SE,95% Conf.,Interval
TP_SEXO,,,,,,
Feminino,739651,42.864,14.170,0.017,42.832,42.897
Masculino,552120,43.469,14.596,0.020,43.431,43.508


In [9]:
ordem_valores = {"TP_SEXO" : ['Feminino', 'Masculino']}
labels = {"TP_SEXO" : "GÊNERO",
         "NT_GER_INTEIRO" : "NOTA_GERAL"}

In [10]:
cores = ['#F04899', '#5187F5']
fig = px.box(df_aluno_info, y="NT_GER_INTEIRO", x = "TP_SEXO", color="TP_SEXO",
             boxmode="overlay", labels = labels, category_orders = ordem_valores,
            width=800, height=400, color_discrete_sequence = cores)

# fig.show()

In [11]:
cores = ['#F04899', '#5187F5']

df_aluno_info = df_aluno_info.sort_values(by = "NT_GER_INTEIRO")
df_aluno_info['count'] = df_aluno_info.groupby(['TP_SEXO', 'NT_GER_INTEIRO'])['NT_GER_INTEIRO'].transform('count')
colors = []

fig = px.line(df_aluno_info, x='NT_GER_INTEIRO', color = 'TP_SEXO', y = "count",
              labels = labels, category_orders = ordem_valores,
            width=800, height=400, color_discrete_sequence = cores)
# fig.show()

## RAÇA

In [12]:
rp.summary_cont(df_aluno_info['NT_GER'].groupby(df_aluno_info['RAÇA']))

,N,Mean,SD,SE,95% Conf.,Interval
RAÇA,,,,,,
Amarela,31191,41.947,14.073,0.080,41.791,42.103
Branca,688299,44.479,14.446,0.017,44.445,44.514
Indígena,4408,39.305,13.966,0.210,38.893,39.718
Não quero declarar,27901,45.882,15.257,0.091,45.703,46.061
Parda,425617,41.307,14.010,0.021,41.265,41.350
Preta,114355,41.508,13.955,0.041,41.427,41.589


In [13]:
ordem_valores = {"RAÇA" : ['Amarela', 'Branca','Indígena','Parda','Preta','Não quero declarar']}
cores = ['#19D3F3', '#FFA15A', '#AB63FA',  '#00CC96', '#EF553B', '#636EFA']

In [14]:
df_aluno_info = df_aluno_info.sort_values(by = "NT_GER_INTEIRO")
df_aluno_info['count'] = df_aluno_info.groupby(['RAÇA','NT_GER_INTEIRO'])['NT_GER_INTEIRO'].transform('count')

fig = px.line(df_aluno_info, x='NT_GER_INTEIRO', color = 'RAÇA', y = "count",
             labels = labels, category_orders = ordem_valores,
            width=800, height=400, color_discrete_sequence = cores)
# fig.show()

In [15]:
fig = px.box(df_aluno_info, y="NT_GER_INTEIRO", x = "RAÇA", color="RAÇA",
             boxmode="overlay", labels = labels, category_orders = ordem_valores,
            width=800, height=400, color_discrete_sequence = cores)

# fig.show()

## IDADE

In [16]:
df_aluno_info.head(3)

,NU_IDADE,TP_SEXO,CO_TURNO_GRADUACAO,ESTADO_CIVIL,RAÇA,NACIONALIDADE,NT_GER,NT_FG,NT_CE,NT_GER_INTEIRO,count
ID_ALUNO_INFO,,,,,,,,,,,
8527,51,Feminino,4,Solteiro(a),Parda,Brasileira,0.000,0.000,0.000,0,481
1632,25,Feminino,3,Casado(a),Branca,Brasileira,0.000,0.000,0.000,0,763
1117,23,Masculino,3,Solteiro(a),Preta,Brasileira,0.000,0.000,0.000,0,170


In [17]:
df_aluno_info[["NU_IDADE"]].describe()

,NU_IDADE
count,1291771.000
mean,28.462
std,7.657
min,16.000
25%,23.000
50%,26.000
75%,32.000
max,87.000


In [18]:
df_aluno_info[["NU_IDADE", "NT_GER"]].corr()

,NU_IDADE,NT_GER
NU_IDADE,1.000,-0.151
NT_GER,-0.151,1.000


In [19]:
df_aluno_info['MEDIA_NOTA_GERAL'] = df_aluno_info.groupby(['NU_IDADE'])['NT_GER'].transform('mean')

In [20]:
labels = { "NU_IDADE" : "IDADE"}
fig = px.scatter(
    df_aluno_info, x='NU_IDADE', y='MEDIA_NOTA_GERAL', opacity=0.65,
    trendline='ols', trendline_color_override='red', labels = labels, width=800, height=400)
# fig.show()

## REGIÃO

In [21]:
df_regiao_tempo = dim_regiao.join(fato_prova.set_index("CO_MUNIC_CURSO")[['ID_TEMPO','NT_GER', 
                                                                           'NT_FG','NT_CE']])
df_tempo = dim_tempo.reset_index()
df_regiao_tempo = df_regiao_tempo.reset_index()
df_regiao_tempo = df_regiao_tempo.merge(df_tempo, how = "left").drop("ID_TEMPO", axis = 1)

In [22]:
rp.summary_cont(df_regiao_tempo['NT_GER'].groupby(df_regiao_tempo['UF']))

,N,Mean,SD,SE,95% Conf.,Interval
UF,,,,,,
AC,3868,40.181,13.360,0.215,39.760,40.602
AL,14968,40.570,14.365,0.117,40.340,40.800
AM,21593,38.519,13.355,0.091,38.341,38.697
AP,4402,39.847,13.097,0.197,39.460,40.234
BA,52239,42.470,14.323,0.063,42.347,42.593
CE,40256,44.612,14.419,0.072,44.471,44.753
DF,27554,45.124,15.363,0.092,44.943,45.305
ES,19406,47.348,14.273,0.102,47.147,47.548
GO,35845,41.882,14.063,0.074,41.737,42.028


In [23]:
rp.summary_cont(df_regiao_tempo['NT_GER'].groupby(df_regiao_tempo['REGIAO_CURSO']))

,N,Mean,SD,SE,95% Conf.,Interval
REGIAO_CURSO,,,,,,
Centro-Oeste,101053,42.307,14.500,0.046,42.217,42.396
Nordeste,245348,42.866,14.362,0.029,42.809,42.923
Norte,78716,40.111,13.608,0.049,40.016,40.206
Sudeste,595012,43.644,14.356,0.019,43.607,43.680
Sul,271642,43.390,14.387,0.028,43.336,43.444


In [24]:
df_regiao_tempo["NT_GER_INTEIRO"] = df_regiao_tempo["NT_GER"].astype(int)

In [25]:
df_regiao_tempo['Count'] = df_regiao_tempo.groupby(['UF', 'ANO'])['NT_GER_INTEIRO'].transform('count')
df_regiao_tempo['MEDIA_NOTA_GERAL'] = df_regiao_tempo.groupby(['UF', 'ANO'])['NT_GER_INTEIRO'].transform('mean')

In [26]:
UF_ordem_alfabetica = list(df_regiao_tempo.UF.sort_values().drop_duplicates())
ano_ordem =  list(df_regiao_tempo.ANO.sort_values().drop_duplicates())
regiao_ordem = list(df_regiao_tempo.REGIAO_CURSO.sort_values().drop_duplicates())

ordem_valores = {"ANO" : ano_ordem,
                 "UF" : UF_ordem_alfabetica,
                 "REGIAO_CURSO" : regiao_ordem}

fig = px.scatter(df_regiao_tempo,  y="MEDIA_NOTA_GERAL", x = "UF" , animation_frame="ANO", animation_group="UF",
                 size="Count", color="REGIAO_CURSO", hover_name="REGIAO_CURSO", range_y=[35,50],
                 category_orders = ordem_valores)
# fig.show()

In [27]:
ordem_valores = {"UF" : UF_ordem_alfabetica}

In [28]:
ordem = ['DF', 'GO', 'MS', 'MT', 'AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN',
       'SE', 'AC', 'AM', 'AP', 'PA', 'RO', 'RR', 'TO', 'ES', 'MG', 'RJ', 'SP',
       'PR', 'RS', 'SC']

df_regiao_tempo.UF = df_regiao_tempo.UF.astype("category")
df_regiao_tempo.UF = df_regiao_tempo.UF.cat.set_categories(ordem)

In [29]:
df_regiao_tempo.head(4)

,CO_MUNIC_CURSO,NOME_MUNICIPIO,UF,REGIAO_CURSO,NT_GER,NT_FG,NT_CE,ANO,NT_GER_INTEIRO,Count,MEDIA_NOTA_GERAL
0,1100023,ARIQUEMES,RO,Norte,16.400,35.500,10.000,2017,16,2370,40.365
1,1100023,ARIQUEMES,RO,Norte,54.800,73.000,48.700,2017,54,2370,40.365
2,1100023,ARIQUEMES,RO,Norte,23.500,37.000,19.000,2017,23,2370,40.365
3,1100023,ARIQUEMES,RO,Norte,34.300,46.400,30.200,2017,34,2370,40.365


In [30]:
ordem = ['DF', 'GO', 'MS', 'MT', 'AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN',
       'SE', 'AC', 'AM', 'AP', 'PA', 'RO', 'RR', 'TO', 'ES', 'MG', 'RJ', 'SP',
       'PR', 'RS', 'SC']

ordem_valores = {"UF" : ordem,
                 "REGIAO_CURSO" : regiao_ordem}

fig = px.box(df_regiao_tempo, y="NT_GER_INTEIRO", x = "UF", color="REGIAO_CURSO",
             boxmode="overlay", labels = labels, width=800, height=400,
             category_orders = ordem_valores)

# fig.show()

In [31]:
ordem = ['DF', 'GO', 'MS', 'MT', 'AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN',
       'SE', 'AC', 'AM', 'AP', 'PA', 'RO', 'RR', 'TO', 'ES', 'MG', 'RJ', 'SP',
       'PR', 'RS', 'SC']

ordem_valores = {"UF" : UF_ordem_alfabetica,
                 "REGIAO_CURSO" : regiao_ordem}

fig = px.box(df_regiao_tempo, y="NT_GER_INTEIRO", x = "UF", color="REGIAO_CURSO",
             boxmode="overlay", labels = labels, width=800, height=400,
             category_orders = ordem_valores)

# fig.show()

In [32]:
fig = px.box(df_regiao_tempo, y="NT_GER_INTEIRO", x = "REGIAO_CURSO", color="REGIAO_CURSO",
             boxmode="overlay", labels = labels, width=800, height=400,
             category_orders = ordem_valores)

# fig.show()

## RENDA

In [33]:
df_economico = dim_aluno_econ.join(fato_prova.set_index("ID_ALUNO_ECONOMICO")[['ID_TEMPO','NT_GER', 
                                                                           'NT_FG','NT_CE']])

In [34]:
# Ordem dos valores na coluna RENDA_FAMILIAR
ordem = ['Até 1,5 salário mínimo',
         'De 1,5 a 3 salários mínimos',
         'De 3 a 4,5 salários mínimos',
         'De 4,5 a 6 salários mínimos',
         'De 6 a 10 salários mínimos',
         'De 10 a 30 salários mínimos',
         'Acima de 30 salários mínimos']

df_economico.RENDA_FAMILIAR = df_economico.RENDA_FAMILIAR.astype("category")
df_economico.RENDA_FAMILIAR = df_economico.RENDA_FAMILIAR.cat.set_categories(ordem)

In [35]:
rp.summary_cont(df_economico['NT_GER'].groupby(df_economico['RENDA_FAMILIAR']))

,N,Mean,SD,SE,95% Conf.,Interval
RENDA_FAMILIAR,,,,,,
"Até 1,5 salário mínimo",258862,39.906,13.807,0.027,39.853,39.959
"De 1,5 a 3 salários mínimos",364140,41.404,13.753,0.023,41.359,41.449
"De 3 a 4,5 salários mínimos",269942,43.074,13.992,0.027,43.021,43.127
"De 4,5 a 6 salários mínimos",143022,44.627,14.245,0.038,44.553,44.700
De 6 a 10 salários mínimos,146657,46.688,14.580,0.038,46.613,46.763
De 10 a 30 salários mínimos,91318,49.725,14.797,0.049,49.629,49.821
Acima de 30 salários mínimos,17830,50.460,15.203,0.114,50.236,50.683


In [36]:
ordem_valores = {"RENDA_FAMILIAR" : ['Até 1,5 salário mínimo',
                             'De 1,5 a 3 salários mínimos',
                             'De 3 a 4,5 salários mínimos',
                             'De 4,5 a 6 salários mínimos',
                             'De 6 a 10 salários mínimos',
                             'De 10 a 30 salários mínimos',
                             'Acima de 30 salários mínimos']}


labels = {"NT_GER_INTEIRO" : "NOTA GERAL",
         "RENDA_FAMILIAR": "RENDA FAMILIAR"}

In [37]:

fig = px.box(df_economico, y="NT_GER", x = "RENDA_FAMILIAR", color="RENDA_FAMILIAR",
             boxmode="overlay", labels = labels, category_orders = ordem_valores, width=800, height=400)

# fig.show()